# Trabalho Prático 2 - Planejamento e Navegação
## Passo 2a: Campos Potenciais para Kuka youbot

### Importação de Bibliotecas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from scipy.ndimage import distance_transform_edt

### Definição dos Parâmetros

O método de Campos Potenciais baseia-se na criação de um campo de forças atrativas e repulsivas.
O objetivo exerce uma força atrativa sobre o robô, enquanto os obstáculos exercem forças repulsivas.

In [ ]:
# Dimensões do mundo
WORLD_WIDTH = 50
WORLD_HEIGHT = 30

# Parâmetros do Kuka youbot
ROBOT_RADIUS = 0.2
SAFETY_MARGIN = 0.1
EFFECTIVE_RADIUS = ROBOT_RADIUS + SAFETY_MARGIN

# Parâmetros do campo potencial
K_ATT = 1.0  # ganho do potencial atrativo
K_REP = 50.0  # ganho do potencial repulsivo
D0 = 2.0  # distância de influência dos obstáculos
STEP_SIZE = 0.5  # tamanho do passo
MAX_ITERATIONS = 1000  # número máximo de iterações
GOAL_THRESHOLD = 0.5  # distância para considerar que chegou ao objetivo

### Carregamento dos Mapas

In [ ]:
# Carregar mapas
mapa1 = mpimg.imread('imagens/mapa1_invertido.png')
mapa2 = mpimg.imread('imagens/mapa2_invertido.png')

### Implementação do Método de Campos Potenciais

O potencial total é a soma do potencial atrativo (em direção ao objetivo) e do potencial repulsivo (afastando dos obstáculos).

**Potencial Atrativo:**
$$U_{att}(q) = \frac{1}{2} k_{att} \cdot d^2(q, q_{goal})$$

**Potencial Repulsivo:**
$$U_{rep}(q) = \begin{cases} 
\frac{1}{2} k_{rep} \left(\frac{1}{d(q, q_{obs})} - \frac{1}{d_0}\right)^2 & \text{se } d(q, q_{obs}) \leq d_0 \\
0 & \text{se } d(q, q_{obs}) > d_0
\end{cases}$$

A força resultante é o negativo do gradiente do potencial total.

In [ ]:
def pixel_to_world(px, py, img_shape, world_width, world_height):
    """Converte coordenadas de pixel para coordenadas do mundo."""
    h, w = img_shape[:2]
    x = (px / w) * world_width
    y = (py / h) * world_height
    return x, y

def world_to_pixel(x, y, img_shape, world_width, world_height):
    """Converte coordenadas do mundo para coordenadas de pixel."""
    h, w = img_shape[:2]
    px = int((x / world_width) * w)
    py = int((y / world_height) * h)
    return px, py

def compute_distance_field(mapa, robot_radius, world_width, world_height):
    """Calcula o campo de distâncias aos obstáculos."""
    # Criar máscara de obstáculos
    obstacles = mapa[:, :, 0] < 0.5
    
    # Calcular distância euclidiana aos obstáculos
    distance_field = distance_transform_edt(~obstacles)
    
    # Converter distância de pixels para metros
    px_per_meter = mapa.shape[1] / world_width
    distance_field = distance_field / px_per_meter
    
    return distance_field

def attractive_force(current, goal, k_att):
    """Calcula a força atrativa em direção ao objetivo."""
    direction = np.array(goal) - np.array(current)
    distance = np.linalg.norm(direction)
    
    if distance < 0.001:
        return np.array([0.0, 0.0])
    
    # Força proporcional à distância
    force = k_att * direction
    
    return force

def repulsive_force(current, distance_field, mapa, k_rep, d0, world_width, world_height):
    """Calcula a força repulsiva dos obstáculos."""
    px, py = world_to_pixel(current[0], current[1], mapa.shape, world_width, world_height)
    
    # Garantir que está dentro dos limites
    px = np.clip(px, 0, distance_field.shape[1] - 1)
    py = np.clip(py, 0, distance_field.shape[0] - 1)
    
    # Distância ao obstáculo mais próximo
    d_obs = distance_field[py, px]
    
    if d_obs > d0:
        return np.array([0.0, 0.0])
    
    # Calcular gradiente da distância (direção de afastamento)
    grad_y, grad_x = np.gradient(distance_field)
    
    gradient = np.array([grad_x[py, px], grad_y[py, px]])
    grad_norm = np.linalg.norm(gradient)
    
    if grad_norm < 0.001:
        return np.array([0.0, 0.0])
    
    gradient = gradient / grad_norm
    
    # Força repulsiva
    force_magnitude = k_rep * (1.0/d_obs - 1.0/d0) * (1.0/(d_obs**2))
    force = force_magnitude * gradient
    
    return force

def potential_field_navigation(start, goal, mapa, k_att, k_rep, d0, step_size, 
                                max_iterations, goal_threshold, world_width, world_height):
    """Executa a navegação por campos potenciais."""
    # Calcular campo de distâncias
    distance_field = compute_distance_field(mapa, EFFECTIVE_RADIUS, world_width, world_height)
    
    # Inicializar trajetória
    path = [start]
    current = np.array(start, dtype=float)
    
    for i in range(max_iterations):
        # Verificar se chegou ao objetivo
        distance_to_goal = np.linalg.norm(current - np.array(goal))
        if distance_to_goal < goal_threshold:
            path.append(tuple(current))
            print(f"Objetivo alcançado em {i} iterações")
            break
        
        # Calcular forças
        f_att = attractive_force(current, goal, k_att)
        f_rep = repulsive_force(current, distance_field, mapa, k_rep, d0, world_width, world_height)
        
        # Força total
        f_total = f_att + f_rep
        
        # Normalizar e aplicar passo
        f_norm = np.linalg.norm(f_total)
        if f_norm < 0.001:
            print(f"Preso em mínimo local na iteração {i}")
            break
        
        direction = f_total / f_norm
        current = current + step_size * direction
        
        # Verificar limites
        current[0] = np.clip(current[0], 0, world_width)
        current[1] = np.clip(current[1], 0, world_height)
        
        path.append(tuple(current))
    
    if distance_to_goal >= goal_threshold:
        print(f"Número máximo de iterações atingido")
    
    return path, distance_field

### Experimento 1: Mapa 1 - Cenário Simples

In [ ]:
# Definir pontos de início e objetivo
start_mapa1 = (5, 15)
goal_mapa1 = (45, 15)

# Executar navegação por campos potenciais
path_mapa1, dist_field_mapa1 = potential_field_navigation(
    start_mapa1, goal_mapa1, mapa1, K_ATT, K_REP, D0, STEP_SIZE,
    MAX_ITERATIONS, GOAL_THRESHOLD, WORLD_WIDTH, WORLD_HEIGHT
)

# Calcular comprimento do caminho
path_length_mapa1 = sum([np.linalg.norm(np.array(path_mapa1[i+1]) - np.array(path_mapa1[i])) 
                          for i in range(len(path_mapa1)-1)])
print(f"Comprimento do caminho: {path_length_mapa1:.2f} m")
print(f"Número de pontos: {len(path_mapa1)}")

### Visualização do Resultado - Mapa 1

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Campo de distâncias
ax1.imshow(dist_field_mapa1, extent=[0, WORLD_WIDTH, 0, WORLD_HEIGHT], 
           origin='lower', cmap='hot')
ax1.axis('off')

# Trajetória
ax2.imshow(mapa1, extent=[0, WORLD_WIDTH, 0, WORLD_HEIGHT], origin='lower')

# Desenhar caminho
path_x = [p[0] for p in path_mapa1]
path_y = [p[1] for p in path_mapa1]
ax2.plot(path_x, path_y, 'r-', linewidth=2)

# Desenhar início e objetivo
ax2.plot(start_mapa1[0], start_mapa1[1], 'go', markersize=10)
ax2.plot(goal_mapa1[0], goal_mapa1[1], 'ro', markersize=10)

ax2.set_xlim(0, WORLD_WIDTH)
ax2.set_ylim(0, WORLD_HEIGHT)
ax2.axis('off')

plt.tight_layout()
plt.savefig('imagens/campos_potenciais_mapa1_youbot.png', dpi=150, bbox_inches='tight')
plt.show()

### Experimento 2: Mapa 2 - Cenário Complexo

In [ ]:
# Definir pontos de início e objetivo
start_mapa2 = (5, 5)
goal_mapa2 = (45, 25)

# Executar navegação por campos potenciais
path_mapa2, dist_field_mapa2 = potential_field_navigation(
    start_mapa2, goal_mapa2, mapa2, K_ATT, K_REP, D0, STEP_SIZE,
    MAX_ITERATIONS, GOAL_THRESHOLD, WORLD_WIDTH, WORLD_HEIGHT
)

# Calcular comprimento do caminho
path_length_mapa2 = sum([np.linalg.norm(np.array(path_mapa2[i+1]) - np.array(path_mapa2[i])) 
                          for i in range(len(path_mapa2)-1)])
print(f"Comprimento do caminho: {path_length_mapa2:.2f} m")
print(f"Número de pontos: {len(path_mapa2)}")

### Visualização do Resultado - Mapa 2

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Campo de distâncias
ax1.imshow(dist_field_mapa2, extent=[0, WORLD_WIDTH, 0, WORLD_HEIGHT], 
           origin='lower', cmap='hot')
ax1.axis('off')

# Trajetória
ax2.imshow(mapa2, extent=[0, WORLD_WIDTH, 0, WORLD_HEIGHT], origin='lower')

# Desenhar caminho
path_x = [p[0] for p in path_mapa2]
path_y = [p[1] for p in path_mapa2]
ax2.plot(path_x, path_y, 'r-', linewidth=2)

# Desenhar início e objetivo
ax2.plot(start_mapa2[0], start_mapa2[1], 'go', markersize=10)
ax2.plot(goal_mapa2[0], goal_mapa2[1], 'ro', markersize=10)

ax2.set_xlim(0, WORLD_WIDTH)
ax2.set_ylim(0, WORLD_HEIGHT)
ax2.axis('off')

plt.tight_layout()
plt.savefig('imagens/campos_potenciais_mapa2_youbot.png', dpi=150, bbox_inches='tight')
plt.show()

### Análise dos Resultados

O método de Campos Potenciais apresenta as seguintes características:

1. **Simplicidade**: O algoritmo é computacionalmente eficiente e fácil de implementar
2. **Comportamento Reativo**: O robô responde em tempo real às forças atrativas e repulsivas
3. **Mínimos Locais**: O método pode ficar preso em mínimos locais, especialmente em ambientes complexos
4. **Suavidade**: As trajetórias geradas são geralmente suaves e contínuas
5. **Parâmetros**: O desempenho depende fortemente dos parâmetros $k_{att}$, $k_{rep}$ e $d_0$

Para o Kuka youbot, que é holonômico, o método é particularmente adequado pois o robô pode seguir
diretamente a direção indicada pelo campo de forças.